# Applied Data Science Capstone Project

IBM Data Sceince Professional Certificate Specialization

In [1]:
import pandas as pd
import numpy as np

## Toronto's Postal Codes
#### Get Wikipedia page's content.
Use requests package to grab Wikipedia page's content to get Toronto's postal codes.

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

try:
    response = requests.get(url)
except requests.exceptions.RequestException as e:
    raise SystemExit(e)
    
if response.status_code != 200:
    raise SystemExit(e)

page = response.content

#### Let's use BeautifulSoup to scrap the data
* Parse recieved content with BeautifulSoup
* Pre-precess response from Wikipedia and create a dataframe with postal codes data with columns **PostalCode**, **Borough**, **Neighborhood**.

credits for hints to https://stackoverflow.com/questions/23377533/python-beautifulsoup-parsing-table

In [3]:
soup = BeautifulSoup(page)
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
table = soup.find('table', attrs={'class':'wikitable'}).find('tbody')

for row in table.find_all('tr'):
    row = row.find_all('td')
    row = [e.text.strip() for e in row]

    # process cells with assigned borough only
    if row == [] or row[1] == 'Not assigned':
        continue
        
    # replace backslashes with commas   
    row[2] = row[2].replace(' /', ',')
    
    # Neighborhood should be the same as the boroughNot if empty
    if row[2] is None:
        row[2] = row[1]
    
    df = df.append(pd.Series(row, index = df.columns), ignore_index=True)
    
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check shape of the dataframe

In [4]:
df.shape

(103, 3)

#### Getting the latitude and the longitude coordinates of each neighborhood.
Use provided in project description csv file **Geospatial_data** to obtain coordinates for postal code.

Load geospatial data.

In [5]:
geospatial_url = 'https://cocl.us/Geospatial_data'
geo_data = pd.read_csv(geospatial_url)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Extend previously created dataframe of postal codes with latitude and longitude coordinates.

In [6]:
df = df.join(geo_data.set_index('Postal Code'), on = 'PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Display Toronto's postal codes on the map.
Install Folium to be able to create maps and create a map.

In [7]:
!pip install folium
import folium
from geopy.geocoders import Nominatim

If map does **not displayed, visit this notebook on IBM Cloud**
https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/ceca7733-0d21-4519-b40f-75ec7cc4b06f/view?access_token=a99205a31c765cbc5a43072f79ca3025cc75c06c8a684ba977dfbd61318071ab

In [28]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
toronto_coords = geolocator.geocode(address)

map_postcodes = folium.Map(
    location=[toronto_coords.latitude, toronto_coords.longitude],
    zoom_start=10
)

# add markers to map
for index, row in df.iterrows():
    label = '{}, {}'.format(row['Neighborhood'], row['Borough'])

    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius = 5,
        popup = folium.Popup(label, parse_html=True),
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False
    ).add_to(map_postcodes)  
    
map_postcodes

## Toronto's Neighborhoods
Let's check boroughs of Toronto.

In [9]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

And pick neighborhoods from boroughs that contain the word **Toronto**.

In [10]:
neigh_list = []
for i, row in df.iterrows():
    if 'Toronto' not in row['Borough']:
        continue
    neigh_list.extend(row['Neighborhood'].split(', '))
print('Number of neighborhoods in Toronto:', len(neigh_list))

Number of neighborhoods in Toronto: 75


Added code to display progress bar, to make output prettier and waiting more comfortable.

https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/

In [11]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

Let's get latitude and longitude coordinates of neighborhoods, there are few neighborhoods for which data are unavailable.

In [12]:
neighborhoods = []
for i, name in enumerate(neigh_list):
    location = geolocator.geocode(name + ', Toronto, Canada')
    if location is not None:
        neighborhoods.append({'Neighborhood': name, 'Latitude': location.latitude, 'Longitude': location.longitude})
    update_progress((i+1) / len(neigh_list))
    
neighborhoods = pd.DataFrame(neighborhoods)
neighborhoods.head()

Progress: [####################] 100.0%


,Latitude,Longitude,Neighborhood
0,43.660706,-79.360457,Regent Park
1,43.640080,-79.380150,Harbourfront
2,43.659659,-79.390340,Queen's Park
3,43.656500,-79.377114,Garden District
4,43.658469,-79.378993,Ryerson


#### Toronto's map of neighborhoods.

In [13]:
map_neighborhoods = folium.Map(
    location=[toronto_coords.latitude, toronto_coords.longitude],
    zoom_start=12
)

for index, row in neighborhoods.iterrows():
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius = 5,
        popup = folium.Popup(row['Neighborhood'], parse_html=True),
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False
    ).add_to(map_neighborhoods)  
    
map_neighborhoods

## Foursquare requests.
Let's get venue data from Foursqare.

In [14]:
# The code was removed by Watson Studio for sharing.

Get venue data for each neighborhood.

In [15]:
radius = 500
limit = 100
venues = []

for i, row in neighborhoods.iterrows():
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, row['Latitude'], row['Longitude'], radius, limit)
    # get results
    results = requests.get(url).json()["response"]['groups'][0]['items']

    for item in results:
        venues.append([
            row['Neighborhood'], row['Latitude'], row['Longitude'],
            item['venue']['name'], item['venue']['location']['lat'], item['venue']['location']['lng'],
            item['venue']['categories'][0]['name']
        ])
    update_progress((i+1) / len(neighborhoods))

venues = pd.DataFrame(venues, columns = [
    'Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
    'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'])
venues.head()

Progress: [####################] 100.0%


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,Regent Park,43.660706,-79.360457,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
2,Regent Park,43.660706,-79.360457,Thai To Go,43.663418,-79.360710,Thai Restaurant
3,Regent Park,43.660706,-79.360457,Sumach Espresso,43.658135,-79.359515,Coffee Shop
4,Regent Park,43.660706,-79.360457,Paintbox Bistro,43.660050,-79.362855,Restaurant


Check venues counts in each neighborhood.

In [16]:
print('Venues dataframe shape:', venues.shape)
print('Unique categories count:', len(venues['Venue Category'].unique()))
venues.groupby('Neighborhood').count().head()

Venues dataframe shape: (3597, 7)
Unique categories count: 297


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,25,25,25,25,25,25
Berczy Park,100,100,100,100,100,100
Brockton,19,19,19,19,19,19
CN Tower,56,56,56,56,56,56


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# group by mean
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,Airport,Airport Service,American Restaurant,Animal Shelter,Antique Shop,Aquarium,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,0.0,0.00,0.00,0.04,0.0,0.00,0.000000,...,0.0,0.01,0.0,0.0,0.000000,0.0,0.01,0.000000,0.0,0.0
1,Bathurst Quay,0.000000,0.0,0.0,0.04,0.04,0.00,0.0,0.00,0.000000,...,0.0,0.00,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0
2,Berczy Park,0.010000,0.0,0.0,0.00,0.00,0.01,0.0,0.01,0.000000,...,0.0,0.01,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0
3,Brockton,0.000000,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.000000,...,0.0,0.00,0.0,0.0,0.157895,0.0,0.00,0.000000,0.0,0.0
4,CN Tower,0.017857,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.035714,...,0.0,0.00,0.0,0.0,0.000000,0.0,0.00,0.017857,0.0,0.0


#### Let's get top 10 venues for neighborhood

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[1:]
    neighborhoods_venues_sorted.iloc[ind, 1:] = row_categories.sort_values(ascending=False).index.values[0:num_top_venues]
    
neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,American Restaurant,Gastropub,Japanese Restaurant,Restaurant,Gym,Cosmetics Shop,Seafood Restaurant,Thai Restaurant
1,Bathurst Quay,Coffee Shop,Café,Park,Gym,Grocery Store,Sushi Restaurant,Caribbean Restaurant,Bank,Sculpture Garden,Ramen Restaurant
2,Berczy Park,Coffee Shop,Italian Restaurant,Café,Restaurant,Hotel,Japanese Restaurant,Cocktail Bar,Bakery,Gastropub,Seafood Restaurant
3,Brockton,Bar,Vietnamese Restaurant,Park,French Restaurant,Gastropub,Dive Bar,Bakery,Jazz Club,Coffee Shop,Café
4,CN Tower,Hotel,Pizza Place,Coffee Shop,Gym,Bar,Baseball Stadium,Ice Cream Shop,Aquarium,Scenic Lookout,Event Space


## Clustering
Let's build k-means model to cluster the neighborhood into 5 clusters.

In [19]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped.drop('Neighborhood', 1))

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 4, 0,
       0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 4, 0, 2, 2, 2, 0, 2, 0, 3, 2,
       2, 1, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0,
       2, 0, 0], dtype=int32)

And combine results with previous data.

In [20]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.660706,-79.360457,Regent Park,0,Coffee Shop,Thai Restaurant,Indian Restaurant,Electronics Store,Food Truck,Beer Store,Sushi Restaurant,Restaurant,Auto Dealership,Pub
1,43.640080,-79.380150,Harbourfront,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Fried Chicken Joint,Pizza Place,Sports Bar,Steakhouse,Brewery
2,43.659659,-79.390340,Queen's Park,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Restaurant,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Thai Restaurant,French Restaurant
3,43.656500,-79.377114,Garden District,0,Clothing Store,Restaurant,Coffee Shop,Hotel,Fast Food Restaurant,Café,Lingerie Store,Bookstore,Japanese Restaurant,Tea Room
4,43.658469,-79.378993,Ryerson,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Diner,Ramen Restaurant,Restaurant,Burger Joint,Japanese Restaurant,Tea Room


#### Create a map
Let's display results of our work on a map.

In [21]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(
    location=[toronto_coords.latitude, toronto_coords.longitude],
    zoom_start=12
)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for i, row in toronto_merged[['Latitude', 'Longitude', 'Neighborhood', 'Cluster Labels']].iterrows():
    label = folium.Popup(str(row['Neighborhood']) + ' Cluster ' + str(row['Cluster Labels']), parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color=rainbow[row['Cluster Labels']-1],
        fill=True,
        fill_color=rainbow[row['Cluster Labels']-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters¶

##### Cluster 1

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,-79.360457,Thai Restaurant,Indian Restaurant,Electronics Store,Food Truck,Beer Store,Sushi Restaurant,Restaurant,Auto Dealership,Pub
1,-79.380150,Café,Hotel,Restaurant,Italian Restaurant,Fried Chicken Joint,Pizza Place,Sports Bar,Steakhouse,Brewery
2,-79.390340,Café,Sandwich Place,Italian Restaurant,Restaurant,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Thai Restaurant,French Restaurant
3,-79.377114,Restaurant,Coffee Shop,Hotel,Fast Food Restaurant,Café,Lingerie Store,Bookstore,Japanese Restaurant,Tea Room
4,-79.378993,Clothing Store,Café,Middle Eastern Restaurant,Diner,Ramen Restaurant,Restaurant,Burger Joint,Japanese Restaurant,Tea Room


##### Cluster 2

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,-79.478313,Pilates Studio,Skating Rink,Women's Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
64,-79.380746,Playground,Bike Trail,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space


##### Cluster 3

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,-79.296712,Pub,Coffee Shop,Breakfast Spot,Japanese Restaurant,Bar,Bakery,Tea Room,Café,Pizza Place
9,-79.418405,Coffee Shop,Cocktail Bar,Sandwich Place,Café,Pub,Indian Restaurant,Japanese Restaurant,Ice Cream Shop,Grocery Store
13,-79.435719,Bakery,Café,Grocery Store,Restaurant,Coffee Shop,Beer Store,Sandwich Place,Vietnamese Restaurant,Mexican Restaurant
14,-79.432636,Brazilian Restaurant,Grocery Store,Pizza Place,Juice Bar,Park,Bus Line,Bar,Restaurant,Coffee Shop
17,-79.394316,Café,Music Venue,Harbor / Marina,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


##### Cluster 4

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,-79.26337,Beer Store,Women's Store,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space


##### Cluster 5

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,-79.383297,Gym,Tennis Court,Park,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
57,-79.413902,Park,Mediterranean Restaurant,Bank,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant


We can check Toronto's neighborhoods groupped by similarity with venueus data using k-Means clustering algorithm.

#### Thank you for attention.

Link to this notebook on IBM Cloud 
https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/ceca7733-0d21-4519-b40f-75ec7cc4b06f/view?access_token=a99205a31c765cbc5a43072f79ca3025cc75c06c8a684ba977dfbd61318071ab